# Exercise: Determing the Extent of Water Bodies

## Overview 
In this exercise, we will create a new notebook to determine the extent of water bodies using the [Water Observation from Space(WOFS) Product](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs).The WOFS product shows water observed for Landsat-8 image.


The notebook will include the following steps:

* Load WOFS and Landsat 8 data
* Identify water pixels from WOFS
* Match landsat to WOFS and plot RGB
* Plot the total number of water pixels in each image, multiplied by the pixel area
* Customise the plot

At the conclusion of this exercise, you will be able to determine the extent of water bodies using the WOFS product.

## Set up notebook

In your Training folder, create a new Python 3 notebook. Name it water_extent_exercise.ipynb. For more instructions on creating a new notebook, see the instructions from [Session 2](../session_2/04_load_data_exercise.ipynb#Make-a-new-notebook).

### Load packages and functions

In the first cell, type the following code and then run the cell to import necessary Python dependencies.


    import matplotlib.pyplot as plt
    %matplotlib inline

    import sys
    import datacube

    sys.path.append('../Scripts')
    from deafrica_datahandling import load_ard, wofs_fuser
    from deafrica_plotting import display_map
    
    
In this exercise, we import one new parameter, `wofs_fuser`.  `wofs_fuser` ensure that the data between scenes is combined correctly for the WOFS


### Connect to the datacube

Enter the following code and run the cell to create our dc object, which provides access to the datacube.

    dc = datacube.Datacube(app="water_extent_exercise")

In the next cell, enter the following code, and then run it to select an area and time.

#### Define the central point of interest
    lat = -6.0873
    lon = 35.1817

    lat_buffer = 0.2
    lon_buffer = 0.2

    #Combine central lat,lon with buffer to get area of interest

    lat_range = (lat - lat_buffer, lat + lat_buffer)
    lon_range = (lon - lon_buffer, lon + lon_buffer)
    
For more information on how the above works, refer to Area of Interest in [Session 5 Exercise](../session_5/02_vegetation_exercise.ipynb#Select-area-of-interest)

#### Define the year
    time = ('2018')

In the next cell, enter the following code, and then run it to show the area on a map. Since we have defined our area using the variables lon_range and lat_range, we can use those instead of typing out `(latitude-buffer, latitude+buffer)` and `(longitude-buffer, longitude+buffer)` again.

    display_map(x=lon_range, y=lat_range)

#### Query Object
    
Notice `lat_range`, `lon_range` and `time` were all defined in the previous cell, so we can use them as variables here. 
Creating an object variable such as the `query` makes it possible to reuse parameters in various functions that accept the same input parameters.

In the next cell, enter the following code, and then run it.

    query = {
        'x': lon_range,
        'y': lat_range,
        'resolution': (-30, 30),
        'output_crs':'EPSG:6933',
        'group_by': 'solar_day',
        'time': (time),
    }

### Load Landsat / WOFs Dataset

In the next cell, enter the following code, and then run it to load the landsat and WOFS dataset to the `ds_landsat` and `ds_wofs`.
From the functions below you can see how we implemented the `query object` simplify the code below.


    ds_landsat = load_ard(dc=dc,
                         products=['ls8_usgs_sr_scene'],
                         measurements=['red', 'green', 'blue'],
                         **query)
                         
    ds_landsat

<img align="middle" src="../_static/session_6/1.PNG" alt="." width="600">

#### Load WOfS

    ds_wofs = dc.load(product=["ga_ls8c_wofs_2"],
             fuse_func=wofs_fuser,
             **query
            )

    ds_wofs
    
<img align="middle" src="../_static/session_6/2.PNG" alt="." width="600">

Comparing the two dataset above, ds_landsat return 44 images whiles ds_wofs returned 46 images. The next cell function select WOFS images at the same timesteps as the landsat. After the processing WOFS images should be the same as the landsat images.

#### select WOfS images at same timesteps as the Landsat data
    ds_wofs_landsat = ds_wofs.sel(time=ds_landsat.time, method='nearest')

#### remove any duplicate time-stamps
    index = np.unique(ds_wofs_landsat.time, return_index=True)
    ds_wofs_landsat = ds_wofs_landsat.isel(time=index[1])
    
    ds_wofs_landsat
    
<img align="middle" src="../_static/session_6/3.PNG" alt="." width="600">

### Understanding the WOFLs

WOFS are stored as a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular feature. Below is a breakdown of which decimal value represent which features.

| Attribute| Decimal value |
|------|----|
| No data | 1|
| Non contiguous |  2 |
| Sea |  4 |
| Terrain or low solar angle |  8 |
| High slope | 16 |
| Cloud shadow | 32 |
| Cloud | 64 |
| Water | 128 |

For example a value 128 indicates that water were observed for the pixel,
whereas a value of 32 would indicate Cloud shadow.

In the next cell we will extract only the water features rom the WOFS products
#### Only show water pixels
    ds_valid_water = ds_wofs_landsat.water == 128

#### Calculate the area per pixel
The number of pixels can be used for the area of the waterbody if the pixel area is known. 

    pixel_length = query["resolution"][1]  # in metres
    m_per_km = 1000  # conversion from metres to kilometres
    area_per_pixel = pixel_length**2 / m_per_km**2

#### Calculate area of water pixels
    ds_valid_water_area = ds_valid_water.sum(dim=['x', 'y']) * area_per_pixel


#### Plot Monthly Time Series

Plotting the Monthly Time series aid us to identify months where the was more and less water within the area of interest.

    plt.figure(figsize=(18, 4))
    ds_valid_water_area.plot(marker='o', color='#9467bd')
    plt.title(f'Time Series Analysis of water observed area')
    plt.xlabel('Dates')
    plt.ylabel('Waterbody area (km$^2$)')
    plt.tight_layout()
    
<img align="middle" src="../_static/session_6/4.PNG" alt="." width="100%">

#### Display of Result for a selected index    
    
    # Plot water extent
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    #plot the true colour image
    rgb(ds_landsat, index=4, ax=ax[0])

    # plot the water extent from WOfS
    ds_wofs_landsat.isel(time=4).water.plot.imshow(ax=ax[1], cmap="Blues", add_colorbar=False)

    # # Titles
    ax[0].set_title("Water Extent - Landsat"), ax[0].xaxis.set_visible(False), ax[0].yaxis.set_visible(False)
    ax[1].set_title("Water Extent -WOFS"), ax[1].xaxis.set_visible(False), ax[1].yaxis.set_visible(False)

    plt.show()
    
<img align="middle" src="../_static/session_6/5.PNG" alt="." width="100%">

## Conclusion

Congratulations! You have made your own water extent notebook.

If you'd like to experiment futher, try running the code with different areas. Did you learn anything interesting to share with us?